In [4]:
%load_ext autoreload
%autoreload 2

# temporary solution for relative imports in case sadl is not installed
# if sadl is installed, no need to use the line
import sys
import os
import inspect
import keras
from inspect import signature
from inspect import signature
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from SADL.time_series.algorithms import tsfedl
from TSFEDL.models_keras import OhShuLih


init_signature = signature(OhShuLih)
# Consider the constructor parameters excluding 'self'
parameters = [p for p in init_signature.parameters.values()
                      if p.name != 'self' and p.kind != p.VAR_KEYWORD]
print(parameters)
kwargs = {"algorithm_": "ohshulih","include_top":False, "label_parser": True}
#model1 = tsfedl.TsfedlAnomalyDetection(**kwargs)





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


RuntimeError: scikit-learn estimators should always specify their parameters in the signature of their __init__ (no varargs). <function OhShuLih at 0x7f8acfa6eac0> with constructor (*args, **kwargs) doesn't  follow this convention.